## Importing sql functions

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
path = "/FileStore/tables/yellow_tripdata_2018_01-1.parquet"
data = spark.read.parquet(path)

In [0]:
display(data.limit(5))

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2018-01-01T00:21:05.000+0000,2018-01-01T00:24:23.000+0000,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.0,0.0,0.3,5.8,null,null
1,2018-01-01T00:44:55.000+0000,2018-01-01T01:03:05.000+0000,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.0,0.0,0.3,15.3,null,null
1,2018-01-01T00:08:26.000+0000,2018-01-01T00:14:21.000+0000,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.0,0.0,0.3,8.3,null,null
1,2018-01-01T00:20:22.000+0000,2018-01-01T00:52:51.000+0000,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.0,0.0,0.3,34.8,null,null
1,2018-01-01T00:09:18.000+0000,2018-01-01T00:27:06.000+0000,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55,null,null


## Query 1. 
Add a column named as "Revenue" into dataframe which is the sum of the below columns 'Fare_amount','Extra','MTA_tax','Improvement_surcharge','Tip_amount','Tolls_amount','Total_amount'

In [0]:
display(data.withColumn("Revenue",
              col('Fare_amount')+ col('Extra')+col('MTA_tax')+col('Improvement_surcharge')+col('Tip_amount')+col('Tolls_amount')+col('Total_amount')).limit(5))

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Revenue
1,2018-01-01T00:21:05.000+0000,2018-01-01T00:24:23.000+0000,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.0,0.0,0.3,5.8,null,null,11.6
1,2018-01-01T00:44:55.000+0000,2018-01-01T01:03:05.000+0000,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.0,0.0,0.3,15.3,null,null,30.6
1,2018-01-01T00:08:26.000+0000,2018-01-01T00:14:21.000+0000,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.0,0.0,0.3,8.3,null,null,16.6
1,2018-01-01T00:20:22.000+0000,2018-01-01T00:52:51.000+0000,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.0,0.0,0.3,34.8,null,null,69.6
1,2018-01-01T00:09:18.000+0000,2018-01-01T00:27:06.000+0000,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55,null,null,33.1


## Query 2.
Increasing count of total passengers in New York City by area

In [0]:
display(data.groupBy('PULocationID', 'DOLocationID').sum('passenger_count').limit(5))

PULocationID,DOLocationID,sum(passenger_count)
79,116,580
246,249,5073
234,144,7047
161,193,100
231,261,5812


## Query 3
Realtime Average fare/total earning amount earned by 2 vendors

In [0]:
display(data.groupBy('VendorID').avg('total_amount'))

VendorID,avg(total_amount)
1,15.127384289902137
2,15.775723474073514


## Query 4
Moving Count of payments made by each payment mode

In [0]:
display(data.groupBy('payment_type').count())

payment_type,count
1,6106416
3,43204
2,2599215
4,11852


## Query 5
Highest two gaining vendor's on a particular date with no of passenger and total distance by cab

In [0]:
display(data.select('VendorID', 'tpep_pickup_datetime', 'passenger_count', 'trip_distance').sort(data.trip_distance.desc()).limit(2))

VendorID,tpep_pickup_datetime,passenger_count,trip_distance
2,2018-01-30T11:41:02.000+0000,1,189483.84
1,2018-01-08T19:44:54.000+0000,0,830.8


## Query 6
Most no of passenger between a route of two location.

In [0]:
display(data.groupBy('PULocationID','DOLocationID').agg(sum('passenger_count').alias('no_of_passenger')).orderBy(col('no_of_passenger').desc()).limit(5))

PULocationID,DOLocationID,no_of_passenger
264,264,186705
237,236,86737
236,236,78691
236,237,71752
237,237,64499


## Query 7
Get top pickup locations with most passengers in last 5/10 seconds.

In [0]:
my_window = Window.partitionBy().orderBy("tpep_pickup_datetime")
df = data.withColumn("prev_value", lag(data.tpep_pickup_datetime).over(my_window))
df = df.withColumn("diff", col('tpep_pickup_datetime').cast('long') - col('prev_value').cast('long')).filter(col('diff').isNotNull())

In [0]:
df = df.withColumn('less_than_10', when(col('diff')<=10, lit(10)).otherwise(lit(20)))

In [0]:
display(df.groupBy('less_than_10', 'PULocationID').agg(sum('passenger_count')).filter(col('less_than_10')==10).orderBy(col('sum(passenger_count)').desc()).limit(5))

less_than_10,PULocationID,sum(passenger_count)
10,161,573841
10,237,572951
10,236,550801
10,230,511445
10,162,490674
